<a href="https://colab.research.google.com/github/jarodchristiansen/Machine-Learning-Deep-Learning/blob/master/Image_Recognition_Prebuilt_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install numpy
!pip install keras
!pip install tensorflow
!pip install matplotlib
!pip install opencv-python
!pip install Image
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.6 MB/s eta 0:00:00
  Created wheel for Image: filename=image-1.5.33-py2.py3-none-any.whl size=19482 sha256=aa7bc8f3f2425353f1603b54ae11bf45f632ee187c064928a7242e8b78615977
  Stored in directory: /root/.cache/pip/wheels/70/0c/a4/7cfa53a5c6225c2db2bfec08e782b43d0f25fdae2e995b69be
Successfully built Image


# Shared Dependencies & Utility Methods

In [51]:
import numpy as np
import matplotlib.pyplot as plt
import os
import keras
import cv2
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.applications import EfficientNetB0, InceptionV3, Xception, VGG16, DenseNet121
from keras.preprocessing import image
from google.colab import files
from PIL import Image

In [44]:
def upload_and_predict(shape=(224, 224)):
  # Upload the image file
  uploaded = files.upload()

  # Assume only one file is uploaded
  for fn in uploaded.keys():
      img_path = fn

  # Load the image
  img = Image.open(img_path)
  img = img.convert('RGB')  # Ensure image is in RGB mode

  # Preprocess the image

  # shape=(None, 299, 299, 3) <- Inception V3 Shapre
  img = img.resize(shape)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  # Make predictions
  preds = model.predict(x)

  # Decode and display predictions
  print('Predicted:', decode_predictions(preds, top=3)[0])

# ResNet50 Image Recognition


In [46]:
# Load the pre-trained ResNet50 model
model = ResNet50(weights='imagenet')

In [47]:
upload_and_predict()

Saving chari.jpg to chari (4).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted: [('n02791124', 'barber_chair', 0.972518), ('n03759954', 'microphone', 0.006426321), ('n04485082', 'tripod', 0.0038684425)]


# EfficientNet

In [48]:
# Load the pre-trained EfficientNetB0 model
model = EfficientNetB0(weights='imagenet')

In [49]:
upload_and_predict()

Saving chari.jpg to chari (5).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted: [('n02791124', 'barber_chair', 0.5739916), ('n04099969', 'rocking_chair', 0.030326502), ('n03109150', 'corkscrew', 0.018104827)]


# InceptionV3

Has to reshape the data to match its expected input, provides shape argument to utility method `upload_and_predict`

In [56]:
model = InceptionV3(weights='imagenet')
upload_and_predict(shape=(299, 299))

Saving chari.jpg to chari (9).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted: [('n04325704', 'stole', 0.99842274), ('n04328186', 'stopwatch', 0.0015737299), ('n01924916', 'flatworm', 3.5943503e-06)]


# Xception

In [54]:
model = Xception(weights='imagenet')

upload_and_predict((299, 299))

Saving moon.webp to moon (2).webp
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted: [('n02769748', 'backpack', 0.96701604), ('n03942813', 'ping-pong_ball', 0.03293608), ('n04026417', 'purse', 9.011787e-06)]


# VGG16

In [57]:
model = VGG16(weights='imagenet')

upload_and_predict()

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


Saving chari.jpg to chari (10).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 839ms/step
Predicted: [('n02791124', 'barber_chair', 0.73793894), ('n04482393', 'tricycle', 0.050473668), ('n04336792', 'stretcher', 0.02236253)]


# DenseNet121

In [58]:
model = DenseNet121(weights='imagenet')

upload_and_predict()

33188688/33188688 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Saving chari.jpg to chari (11).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Predicted: [('n06359193', 'web_site', 0.99963164), ('n03729826', 'matchstick', 0.00021987711), ('n04286575', 'spotlight', 0.00014853699)]


# All Model Evaluation

In [59]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# def load_dataset(dataset_path, target_size=(224, 224), batch_size=32):
#     datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
#     dataset = datagen.flow_from_directory(dataset_path, target_size=target_size, batch_size=batch_size, class_mode='categorical', shuffle=False)
#     return dataset


import tensorflow as tf
import tensorflow_datasets as tfds

def load_labeled_dataset(name='cifar10'):
    # Load the dataset
    (train_ds, test_ds), ds_info = tfds.load(
        name,
        split=['train', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )

    # Normalize and resize the images if necessary
    def preprocess_image(image, label):
        image = tf.image.resize(image, (224, 224))
        image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1] range
        return image, label

    train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_ds = test_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Batch and prefetch the data for performance
    train_ds = train_ds.cache().batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    test_ds = test_ds.cache().batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return train_ds, test_ds, ds_info

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.applications import ResNet50, EfficientNetB0, InceptionV3, Xception, VGG16, DenseNet121
import numpy as np
import tensorflow as tf

# List of models
models = [
    ('ResNet50', ResNet50(weights='imagenet')),
    ('EfficientNetB0', EfficientNetB0(weights='imagenet')),
    # ('InceptionV3', InceptionV3(weights='imagenet')),
    # ('Xception', Xception(weights='imagenet')),
    ('VGG16', VGG16(weights='imagenet')),
    ('DenseNet121', DenseNet121(weights='imagenet'))
]

# Load the dataset
train_ds, test_ds, ds_info = load_labeled_dataset()

# Initialize arrays to store metrics
accuracies, precisions, recalls, f1_scores = [], [], [], []

# Loop over each model
for name, model in models:
    y_true = []
    y_pred = []

    # Loop over each batch in the test dataset
    for X_batch, y_batch in test_ds:
        # Append true labels to y_true
        y_true.extend(y_batch.numpy())

        # Make predictions
        preds = model.predict(X_batch)

        # Append predicted labels to y_pred
        y_pred_batch = np.argmax(preds, axis=1)
        y_pred.extend(y_pred_batch)

    # Convert lists to numpy arrays for metric calculations
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    # Store metrics
    accuracies.append((name, accuracy))
    precisions.append((name, precision))
    recalls.append((name, recall))
    f1_scores.append((name, f1))

# Print the results
for name, accuracy in accuracies:
    print(f"Model: {name} | Accuracy: {accuracy}")
for name, precision in precisions:
    print(f"Model: {name} | Precision: {precision}")
for name, recall in recalls:
    print(f"Model: {name} | Recall: {recall}")
for name, f1 in f1_scores:
    print(f"Model: {name} | F1-Score: {f1}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
